In [2]:
import numpy as np
import sympy as sp
import pandas as pd
from numpy.linalg import norm

In [3]:
def levenberg_marquardt(func, indep_vars_symbols, params_symbols, indep_vars_values, y_vec, init_vals, tol = 1e-4, max_iter = 10, lambda_0 = 10):
    
    # Obtenemos la lista de funciones evaluados en todos los puntos
    # de variables independientes
    f = []
    for idx in range(len(indep_vars_values[indep_vars_symbols[0]])):
        subs_values = {key: values[idx] for key, values in indep_vars_values.items()}
        f.append(func.subs(subs_values))
    # Residuos
    r = [f_j - y_j for f_j, y_j in zip(f, y_vec)]
    # Jacobiano
    jacobo = sp.Matrix(f).jacobian(params_symbols)

    # Inicializamos variables para el bucle
    results = []
    params_k = init_vals
    lambda_k = lambda_0
    diff_cond = 2 * tol
    iter_count = 0
    last_iter = False

    while iter_count < max_iter:

        # Valores de la iteración k
        jacobo_k = jacobo.subs(params_k)
        r_k = np.asarray([r_j.subs(params_k) for r_j in r], dtype = np.float16)
        d_k = - np.dot(
            sp.Matrix(
                np.dot(jacobo_k.T, jacobo_k) +
                lambda_k * np.diag(np.dot(jacobo_k.T, jacobo_k).diagonal())).inv(),
            np.dot(jacobo_k.T, r_k))
        
        # Valores de la iteración k + 1
        params_k1 = {key_j: value_j + d_k_j for key_j, value_j, d_k_j in zip(params_k.keys(), params_k.values(), d_k)}
        r_k1 = np.asarray([r_j.subs(params_k1) for r_j in r], dtype = np.float16)

        # Añadimos la iteración a la lista de resultados
        results.append([iter_count] + list(params_k.values()) + [sum(r_k**2), lambda_k] + list(d_k))

        # Condición para el siguiente paso
        lambda_cond = norm(r_k1) ** 2 - norm(r_k)**2
        if lambda_cond > 0:
            lambda_k *= 10
        else:
            lambda_k /= 10
            params_k = params_k1
            diff_cond = lambda_cond
        iter_count += 1

        # Condición de salida
        if last_iter:
            break
        if abs(diff_cond) < tol:
            last_iter = True

    # Generamos tabla con los resultados
    table = pd.DataFrame(results)
    table.columns = ['Iter'] + [str(symbol) for symbol in params_symbols] + ['\u2211 r²', 'lambda'] + ['d_' + str(symbol) for symbol in params_symbols]
    
    return table, params_k

In [8]:
# Ejemplo diapositivas

# Datos
# Variables independientes
t = sp.symbols('t')
indep_vars_symbols = [t]
indep_vars_values = {t: [0, 0.5, 1, 1.5, 2, 2.5, 3]}
y_vec = [1.145, 0.512, 0.401, 0.054, 0.038, 0.014, 0.046]

# Ecuación de ajuste
# Parámetros
x = sp.symbols('x')
params_symbols = [x]
func = np.e ** (- x * t)

# Valores iniciales para los parámetros
init_vals = {x: 4}

# Llamamos a la función
results_table, x_sol = levenberg_marquardt(func, indep_vars_symbols, params_symbols, indep_vars_values, y_vec, init_vals)
results_table

,Iter,x,∑ r²,lambda,d_x
0,0,4,0.315738,10.000,-0.603468900142443
1,1,3.39653109985756,0.270327,1.000,-2.24095234867595
2,2,1.15557875118161,0.051601,0.100,0.102895790603077
3,3,1.25847454178469,0.047835,0.010,0.0117833752674878
4,4,1.27025791705218,0.047798,0.001,-0.000502339975430241


In [6]:
# Práctica 1: Ejercicio 2

# Datos
# Variables independientes
t = sp.symbols('t')
indep_vars_symbols = [t]
indep_vars_values = {t: [1, 2, 3, 4, 5, 6, 7, 8]}
y_vec = [8.3, 11.0, 14.7, 19.7, 26.7, 35.2, 44.4, 55.9]

# Ecuación de ajuste
# Parámetros
x1, x2 = sp.symbols('x1 x2')
params_symbols = [x1, x2]
func = x1 * np.e ** (x2 * t)

# Valores iniciales para los parámetros
init_vals = {x1: 4, x2: 0.3}

# Llamamos a la función
results_table, x_sol = levenberg_marquardt(func, indep_vars_symbols, params_symbols, indep_vars_values, y_vec, init_vals)
results_table

,Iter,x1,x2,∑ r²,lambda,d_x1,d_x2
0,0,4,0.3,562.156250,10.000000,0.118266385911159,0.00386562305474749
1,1,4.11826638591116,0.303865623054747,408.375000,1.000000,0.415786462744169,0.0110958163472266
2,2,4.53405284865533,0.314961439401974,94.598999,0.100000,0.505393525992390,-0.00700339166296091
3,3,5.03944637464772,0.307958047739013,47.090332,0.010000,1.25608472026452,-0.0343769776532678
4,4,6.29553109491224,0.273581070085745,15.015185,0.001000,0.685670547427508,-0.0118541674805405
5,5,6.98120164233974,0.261726902605205,6.214142,0.000100,0.0184113391204312,0.000362651619321495
6,6,6.99961298146018,0.262089554224526,6.011108,0.000010,0.000536560280680483,-1.23438109768545e-5
7,7,7.00014954174086,0.262077210413549,6.012024,0.000001,3.27353664969455e-5,-9.41828673279203e-7
